In [238]:
import pandas
import unicodedata
import re

In [239]:
# CSV files exported from the XLSX document
main_table = pandas.read_csv('_hlavni.csv', header=None, encoding='iso-8859-2')
item_table = pandas.read_csv('_polozky.csv', encoding='iso-8859-2')

In [240]:
regions = main_table.loc[1:20, 0:1]
regions = regions[regions[1] != 'prázdný soubor']
regions

,0,1
1,00.csv,nehody na území hl. m. Prahy
2,01.csv,nehody na území Středočeského kraje
3,02.csv,nehody na území Jihočeského kraje
4,03.csv,nehody na území Plzeňského kraje
5,04.csv,nehody na území Ústeckého kraje
6,05.csv,nehody na území Královéhradeckého kraje
7,06.csv,nehody na území Jihomoravského kraje
8,07.csv,nehody na území Moravskoslezského kraje
15,14.csv,nehody na území Olomouckého kraje
16,15.csv,nehody na území Zlínského kraje


In [241]:
def slugify(string):
    """Convert string to an "identifier" consisting of only a-z and underscore"""
    if pandas.isnull(string):
        return string
    letters = ''.join(
        c for c in unicodedata.normalize('NFD', string).lower()
        if unicodedata.category(c)[0] in 'LPSNZ'
    )
    return re.sub('[^a-z0-9]+', '_', letters).strip('_')

assert slugify('ABC') == 'abc'
assert slugify(' á B, č ') == 'a_b_c', slugify(' á B, č ')

In [275]:
row_ids = {}
decode_key = {}
for index, row in item_table.iterrows():
    item = row['Položka']
    if not pandas.isnull(row['Formulář DN']):
        current_entry = slugify(row['Popis'])
        current_id = item
    elif isinstance(item, str) and re.match('[a-z]', item):
        current_entry = slugify(row['Popis'])
        current_id = item
    elif isinstance(item, str) and '-' in item:
        start, stop = item.split('-')
        for i in range(int(start), int(stop)+1):
            decode_key.setdefault(current_entry, {})[i] = row['Popis'].strip()
    elif isinstance(item, str) and item.isnumeric():
        number = int(item)
        decode_key.setdefault(current_entry, {})[number] = row['Popis'].strip()
        if (
            not pandas.isnull(row['Bližší definice'])
            and current_id != 'p45a'  # the detail field is not unique for car manufacturers
        ):
            decode_key[current_entry][number] += '; ' + row['Bližší definice'].strip()
    else:
        print('ignore row:', index, item)
    row_ids[current_id] = current_entry
    last_detail = row['Bližší definice']


ignore row: 0 nan
ignore row: 66 NEPŘIMĚŘENÁ RYCHLOST JÍZDY
ignore row: 76 NESPRÁVNÉ PŘEDJÍŽDĚNÍ
ignore row: 88 NEDÁNÍ PŘEDNOSTI V JÍZDĚ
ignore row: 103 NESPRÁVNÝ ZPŮSOB JÍZDY
ignore row: 120 TECHNICKÁ ZÁVADA VOZIDLA
ignore row: 325 nan
ignore row: 326 nan
ignore row: 327 nan
ignore row: 328 nan
ignore row: 329 nan
ignore row: 475 nan
ignore row: 476 nan
ignore row: 525 nan
ignore row: 555 nan


In [276]:
main_columns = [row_ids.get(id, id) for id in hlavni.loc[30]]
main_columns

['identifikacni_cislo',
 'druh_pozemni_komunikace',
 'cislo_pozemni_komunikace',
 'den_mesic_rok',
 'weekday(p2a)',
 'cas',
 'druh_nehody',
 'druh_srazky_jedoucich_vozidel',
 'druh_pevne_prekazky',
 'charakter_nehody',
 'zavineni_nehody',
 'alkohol_u_vinika_nehody_pritomen',
 'hlavni_priciny_nehody',
 'usmrceno_osob',
 'tezce_zraneno_osob',
 'lehce_zraneno_osob',
 'celkova_hmotna_skoda',
 'druh_povrchu_vozovky',
 'stav_povrchu_vozovky_v_dobe_nehody',
 'stav_komunikace',
 'povetrnostni_podminky_v_dobe_nehody',
 'viditelnost',
 'rozhledove_pomery',
 'deleni_komunikace',
 'situovani_nehody_na_komunikaci',
 'rizeni_provozu_v_dobe_nehody',
 'mistni_uprava_prednosti_v_jizde',
 'specificka_mista_a_objekty_v_miste_nehody',
 'smerove_pomery',
 'pocet_zucastnenych_vozidel',
 'misto_dopravni_nehody',
 'druh_krizujici_komunikace',
 'druh_vozidla',
 'vyrobni_znacka_motoroveho_vozidla',
 'rok_vyroby_vozidla',
 'charakteristika_vozidla',
 'smyk',
 'vozidlo_po_nehode',
 'unik_provoznich_prepravovanych

In [277]:
pedestrian_columns = [row_ids.get(id, id) for id in hlavni.loc[35] if not pandas.isnull(id)]
pedestrian_columns

['identifikacni_cislo',
 'kategorie_chodce',
 'stav_chodce',
 'chovani_chodce',
 'situace_v_miste_nehody']

In [278]:
import pprint
with open('explanations.py', 'w') as file:
    print('# Zdroj: https://www.policie.cz/clanek/statistika-nehodovosti-900835.aspx?q=Y2hudW09Mg%3d%3d', file=file)

    print(file=file)

    print('# Význam souborů 00.csv, 01.csv, ...  (chybějící jsou prázdné)', file=file)
    print('file_names = ', end='', file=file)
    pprint.pprint(dict(hlavni_kraje.values), stream=file)

    print(file=file)

    print('# Jména sloupců v souborech 00.csv, 01.csv, ...', file=file)
    print('main_columns = ', end='', file=file)
    pprint.pprint(main_columns, stream=file)

    print(file=file)

    print('# Jména sloupců v souboru CHODCI.csv', file=file)
    print('ped_columns = ', end='', file=file)
    pprint.pprint(pedestrian_columns, stream=file)

    print(file=file)

    print('# Význam čísel v jednotlivých sloupcích', file=file)
    print('decode_key = ', end='', file=file)
    pprint.pprint(decode_key, stream=file)


In [272]:
with open('explanations.py') as file:
    print(file.read())

# Zdroj: https://www.policie.cz/clanek/statistika-nehodovosti-900835.aspx?q=Y2hudW09Mg%3d%3d

# Význam souborů 00.csv, 01.csv, ...  (chybějící jsou prázdné)
file_names = {'00.csv': 'nehody na území hl. m. Prahy',
 '01.csv': 'nehody na území Středočeského kraje',
 '02.csv': 'nehody na území Jihočeského kraje',
 '03.csv': 'nehody na území Plzeňského kraje',
 '04.csv': 'nehody na území Ústeckého kraje',
 '05.csv': 'nehody na území Královéhradeckého kraje',
 '06.csv': 'nehody na území Jihomoravského kraje',
 '07.csv': 'nehody na území Moravskoslezského kraje',
 '14.csv': 'nehody na území Olomouckého kraje',
 '15.csv': 'nehody na území Zlínského kraje',
 '16.csv': 'nehody na území kraje Vysočina',
 '17.csv': 'nehody na území Pardubického kraje',
 '18.csv': 'nehody na území Libereckého kraje',
 '19.csv': 'nehody na území Karlovarského kraje'}

# Jména sloupců v souborech 00.csv, 01.csv, ...
main_columns = ['identifikacni_cislo',
 'druh_pozemni_komunikace',
 'cislo_pozemni_komunikace',
 'den_